# Что такое middleware и каковы его основные задачи в веб-приложениях
Middleware в веб-приложениях — это промежуточный слой, который обрабатывает запросы и ответы перед тем, как они достигнут обработчиков маршрутов или после.

1. **Middleware** — это функции или объекты, которые выполняются между запросом и ответом в веб-приложениях.
2. Основные задачи:

   * **Обработка запросов**: Проводят проверки или модификации запроса перед его передачей в обработчик (например, аутентификация, валидация данных).
   * **Обработка ответов**: Модифицируют ответ, прежде чем он будет отправлен клиенту (например, добавление заголовков, логирование).
   * **Обработка ошибок**: Ловят исключения, которые могут возникнуть в процессе обработки запроса, и корректно их обрабатывают.
   * **Логирование**: Фиксируют данные о запросах и ответах для отладки или мониторинга.
3. В веб-фреймворках, таких как FastAPI или Django, middleware можно настраивать для глобальной обработки запросов и ответов.


## Middleware FastAPI
В [FastAPI](https://fastapi.tiangolo.com/advanced/middleware/#gzipmiddleware) есть несколько встроенных middleware, которые обеспечивают базовую функциональность, такую как логирование, обработка CORS и управление сессиями.

1. **`CORSMiddleware`**: Обрабатывает заголовки CORS (Cross-Origin Resource Sharing), позволяя контролировать доступ к API с других доменов.
2. **`BaseHTTPMiddleware`**: Основной класс для создания собственных middleware, который позволяет работать с запросами и ответами на более низком уровне.
3. **`GZipMiddleware`**: Сжимает ответы с помощью GZIP для уменьшения их размера и ускорения передачи.
4. **`TrustedHostMiddleware`**: Проверяет, что запросы приходят с допустимых хостов (профилактика атак с подменой хоста).
5. **`SessionMiddleware`**: Используется для работы с сессиями (например, для хранения данных о пользователе между запросами).
6. **`LoggingMiddleware`**: Встраивает базовое логирование всех запросов и ответов.

Эти middleware позволяют быстро интегрировать стандартные решения в приложение.

## Синтаксис
Инициализируем `FastAPI`. Вызываем от приложения метод `add_middleware()` с классом миддлы, нужной нам.

### `TrustedHostMiddleware`
Проверяет заголовок `Host` на соответствие вайт-листу. Иначе отлуп.

In [5]:
import time

from fastapi import FastAPI, Request
from numpy.ma.core import minimum
from starlette.middleware.trustedhost import TrustedHostMiddleware
from starlette.responses import Response
from uvicorn import Server, Config
from watchfiles import awatch

app = FastAPI()

app.add_middleware(TrustedHostMiddleware, allowed_hosts=['example.com'])

@app.get('/')
async def root():
    """GET http://127.0.0.1:8000/ В Заголовке должен быть передан Host: example.com, иначе не пустит"""
    return {'msg': 'hi'}

if __name__ == "__main__":
    config = Config(app)
    server = Server(config)
    await server.serve()

INFO:     Started server process [52258]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56855 - "POST / HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:56855 - "GET / HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:56865 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [52258]


## `HTTPSRedirectMiddleware`
Редиректит с `http` на `https`.

In [11]:
from starlette.middleware.httpsredirect import HTTPSRedirectMiddleware

app = FastAPI()
app.add_middleware(HTTPSRedirectMiddleware)

@app.get('/')
async def root():
    """GET http://127.0.0.1:8000/ перебросит на https://127.0.0.1:8000"""
    return {'msg': 'hi'}

if __name__ == "__main__":
    config = Config(app)
    server = Server(config)
    await server.serve()

INFO:     Started server process [52258]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56921 - "GET / HTTP/1.1" 307 Temporary Redirect


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [52258]


## `GZipMiddleware`
Сжимает ответ, если в запросе есть заголовок `Accept-Encoding: gzip`.

In [16]:
from starlette.middleware.gzip import GZipMiddleware

app = FastAPI()
app.add_middleware(GZipMiddleware, minimum_size=100, compresslevel=9)

@app.get('/')
async def root():
    """GET http://127.0.0.1  в ответе заголовок Content-Length будет меньше из-за GZip сжатия"""
    return 'somethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbigsomethingbig'

if __name__ == "__main__":
    config = Config(app)
    server = Server(config)
    await server.serve()

INFO:     Started server process [52258]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57918 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:57918 - "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [52258]


## Кастомный middleware

### Через декоратор
Простой, менее гибгий. Делаем декоратор от `app` с методом `middleware`. Указываем единственный поддерживаемый тип `http`. Принимает аргументами `request` и `call_next`. Функция `call_next` вызывается после выполнения логики над запросом и возвращает ответ. После обрабатываем логику ответа. Например, добавляем хедер, меняем содержимое, логируем и тд. Возвращаем ответ.

In [9]:
import time

from fastapi import FastAPI, Request
from uvicorn import Server, Config

app = FastAPI()

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    """Возвращает в любом запросе заголовок X-Process-Time с замером времени исполнения."""
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["X-Process-Time"] = str(process_time)
    return response

@app.get('/')
async def root():
    """GET http://127.0.0.1:8000/ В ответе есть заголовок X-Process-Time."""
    return {'msg': 'hi'}

if __name__ == "__main__":
    config = Config(app)
    server = Server(config)
    await server.serve()

NameError: name 'Request' is not defined

### Подход с классом `BaseHTTPMiddleware'
Класс из библиотеки `Starlette`. Наследуемся, определяем логику метода `dispatch()`. Указываем те же два аргумента: `request` и `call_next`. Если нужны атрибуты у класса, переопределить `__init__` и обязательно сделать первым аргументом `app`.

In [10]:
from starlette.middleware.base import BaseHTTPMiddleware

from fastapi import FastAPI, Request
from uvicorn import Server, Config

class MyMiddleware(BaseHTTPMiddleware):
    def __init__(self, app, val: str):
        super().__init__(app)
        self.val = val

    async def dispatch(self, request: Request, call_next):
        print(f'Значение присвоенное классу миддла {self.val}')
        content_type = request.headers.get('Content-Type')
        print(f'Content-Type: {content_type}')

        response = await call_next(request)
        response.headers['Custom'] = 'some-val'

        return response

app = FastAPI()
app.add_middleware(MyMiddleware, val='abc')

@app.get('/')
async def root():
    """GET http://127.0.0.1:8000/ Печатает атрибут val миддла, и заголосов Content-Type запроса.

     В ответе добавляет заголвоок Custom."""
    return {'msg': 'hi'}

if __name__ == "__main__":
    config = Config(app)
    server = Server(config)
    await server.serve()

INFO:     Started server process [62167]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Значение присвоенное классу миддла abc
Content-Type: None
INFO:     127.0.0.1:65136 - "GET / HTTP/1.1" 200 OK
Значение присвоенное классу миддла abc
Content-Type: None
INFO:     127.0.0.1:65137 - "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 Not Found
Значение присвоенное классу миддла abc
Content-Type: None
INFO:     127.0.0.1:65144 - "GET / HTTP/1.1" 200 OK
Значение присвоенное классу миддла abc
Content-Type: None
INFO:     127.0.0.1:65144 - "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 Not Found
Значение присвоенное классу миддла abc
Content-Type: None
INFO:     127.0.0.1:65144 - "GET / HTTP/1.1" 200 OK
Значение присвоенное классу миддла abc
Content-Type: None
INFO:     127.0.0.1:65144 - "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [62167]
